# Data Scientist Professional Practical Exam Submission

**Use this template to write up your summary for submission. Code in Python or R needs to be included.**


# Data Validation 

our data contains 947 rows and 8 columns before cleaning and validataion. I have validated all the columns against the criteria in the dataset table:

- recipe: 947 unique values without missing values. this column act as index for each recipe
- calories: 895 unique values with 52 missing values, 
- carbohydrate: 895 unique values with 52 missing values
- sugar: 895 unique values with 52 missing values
- protein: 895 unique values with 52 missing values
- category: 11 unique values without missing values
- servings: 6 unique value with no missing values but we need to explore this because the data is not numerical 
- high_traffic: 1 unique value with 373 missing values. the missing values are the low traffic we need to fix this

## Steps I will take to validate and clean the data:

**1. handling missing values:** I checked for missing values in the dataset and handled them appropriately.

**2. handling outliers:** I used various techniques to detect and handle outliers in the dataset.

**3. Checking for skewness:** I checked the distribution of the data in each column and applied log transformation when necessary to make the data more normally distributed.

**4. Checking for correlation:** I used visualiziation and statistical methods to check the correlation between the different columns of the data.

**5. Adding new features:** making new features depending on the information gained from the data to supports our insights




In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency,ttest_ind
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,roc_curve,precision_score, recall_score,f1_score
import keras
from keras.layers import Dense
from keras.models import Sequential




2023-01-26 17:23:44.656900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 17:23:44.656922: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
df=pd.read_csv("recipe_site_traffic_2212.csv")
print(df.shape)
df.head()

(947, 8)


,recipe,calories,carbohydrate,sugar,protein,category,servings,high_traffic
0,1,NaN,NaN,NaN,NaN,Pork,6,High
1,2,35.48,38.56,0.66,0.92,Potato,4,High
2,3,914.28,42.68,3.09,2.88,Breakfast,1,NaN
3,4,97.03,30.56,38.63,0.02,Beverages,4,High
4,5,27.05,1.85,0.80,0.53,Beverages,4,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   recipe        947 non-null    int64  
 1   calories      895 non-null    float64
 2   carbohydrate  895 non-null    float64
 3   sugar         895 non-null    float64
 4   protein       895 non-null    float64
 5   category      947 non-null    object 
 6   servings      947 non-null    object 
 7   high_traffic  574 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 59.3+ KB


In [6]:
df.describe(include='all')

,recipe,calories,carbohydrate,sugar,protein,category,servings,high_traffic
count,947.000000,895.000000,895.000000,895.000000,895.000000,947,947,574
unique,NaN,NaN,NaN,NaN,NaN,11,6,1
top,NaN,NaN,NaN,NaN,NaN,Breakfast,4,High
freq,NaN,NaN,NaN,NaN,NaN,106,389,574
mean,474.000000,435.939196,35.069676,9.046547,24.149296,NaN,NaN,NaN
std,273.519652,453.020997,43.949032,14.679176,36.369739,NaN,NaN,NaN
min,1.000000,0.140000,0.030000,0.010000,0.000000,NaN,NaN,NaN
25%,237.500000,110.430000,8.375000,1.690000,3.195000,NaN,NaN,NaN
50%,474.000000,288.550000,21.480000,4.550000,10.800000,NaN,NaN,NaN
75%,710.500000,597.650000,44.965000,9.800000,30.200000,NaN,NaN,NaN


## Now fixing the high_traffic

The high traffic variable is the target variable. It has only one value which is **"High"** so this obvious the missing values in that case are the low traffic data so I will fill the missing values with **"Low"**

In [7]:
df['high_traffic'].fillna('Low',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   recipe        947 non-null    int64  
 1   calories      895 non-null    float64
 2   carbohydrate  895 non-null    float64
 3   sugar         895 non-null    float64
 4   protein       895 non-null    float64
 5   category      947 non-null    object 
 6   servings      947 non-null    object 
 7   high_traffic  947 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 59.3+ KB


## Fixing Nutritions columns

It appears from the data that calories, carbohydrate, sugar and protein have the same number of missing value 52 When I checked below I found that they are missing from the same rows so we have 52 recipe without any nutritional information.<br>

Also I checked if they are missing from the same category but no there is missing values in most of the categories.

### handling method
It is logical that The recipes in the same category with the same number of servings tends to have similar nutritions. For example meat in most cases will have the high amount of protien and desserts will have high calories that is a common sense.

- group the data with category and servings and the aggregate function is the mean 
- fill the Null values with the mean from the grouped data depending on the category of the recipe and the serving 




Checking rows that has missing values

In [8]:
df[df['calories'].isna()].count()

recipe          52
calories         0
carbohydrate     0
sugar            0
protein          0
category        52
servings        52
high_traffic    52
dtype: int64

checking if the missing values are in the same category

In [9]:
df[df['calories'].isna()].pivot_table(index=['high_traffic','category'],values='recipe',aggfunc='count')

recipe
high_traffic category              
High         Chicken              2
             Chicken Breast       2
             Dessert              5
             Lunch/Snacks         5
             Meat                 3
             One Dish Meal        1
             Pork                11
             Potato               5
             Vegetable            5
Low          Chicken              3
             Chicken Breast       2
             Dessert              1
             Lunch/Snacks         2
             Meat                 2
             One Dish Meal        3

create a pivot table that groups the data by "category" and "servings" and calculates the mean for each of the columns "calories", "carbohydrate", "protein", and "sugar" for each group.

In [10]:
means=df.pivot_table(index=["category","servings"],values=['calories','carbohydrate','protein','sugar'])
means.head(10)

calories  carbohydrate    protein      sugar
category  servings                                                
Beverages 1         180.341000     13.237000   0.751500  10.337500
          2         245.478824     21.030000   0.572353  15.809412
          4         162.894706     14.629118   0.551765  16.265294
          6         144.427619     16.586190   0.533333   5.947619
Breakfast 1         401.969615     39.189615  24.025769   4.534231
          2         374.082857     41.820000  12.289048   7.759048
          4         266.214634     40.415122  18.059756   7.748780
          6         330.973889     36.403889   9.681667  11.222778
Chicken   1         419.004000     27.174667  46.293333   4.266667
          2         573.600000     53.863846  55.700769   4.430769

now filling the missing values with the means 

In [11]:
df[['calories','carbohydrate','sugar','protein']] = df.groupby(['category','servings']).apply(lambda x: x[['calories','carbohydrate','sugar','protein']].fillna(means.loc[x.name]))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   recipe        947 non-null    int64  
 1   calories      947 non-null    float64
 2   carbohydrate  947 non-null    float64
 3   sugar         947 non-null    float64
 4   protein       947 non-null    float64
 5   category      947 non-null    object 
 6   servings      947 non-null    object 
 7   high_traffic  947 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 59.3+ KB


## Fixing The Servings column

The problem with servings column it should be numerical but it appears here as an object.<br>
after exploring the column I found that it contains 2 outliers values "4 as a snack" which appearead twice and "6 as snack" which appeared once.<br>

- I droped them from the data because they represent negligible percentage.
- I changed the datatype of the column into integer instead of object


In [12]:
df[(df['servings']=='4 as a snack') | (df['servings']=='6 as a snack')]

,recipe,calories,carbohydrate,sugar,protein,category,servings,high_traffic
451,452,1542.53,14.73,5.71,29.92,Lunch/Snacks,4 as a snack,High
548,549,1440.69,30.59,5.78,30.63,Lunch/Snacks,6 as a snack,High
735,736,513.75,65.87,18.63,18.39,Lunch/Snacks,4 as a snack,Low


In [13]:
df = df[~((df['servings']=='4 as a snack') | (df['servings']=='6 as a snack'))]
df['servings']=df['servings'].astype(np.int64)

# Data Exploration and Analysis

Now that the data has been cleaned, it is important to explore it in order to gain valuable insights. The goals of this analysis include:

**1. Identifying outliers:** Through data exploration, I will identify any potential outliers that may need to be handled as part of the data validation process.

**2. Assessing skewness:** Numerical data will be evaluated for skewness, as this can have an impact on the model building process.

**3. Examining correlation:** The relationships between different features will be analyzed to identify any correlations that may exist.

**4. Investigating association:** Given that this is a classification problem, it is important to determine the strength of the association between independent variables and the dependent variable.

**5. Creating new features:** I will examine the data to see if new features can be extracted that will support the solution.

visualize the distribution of recipes across categories,servings and high_traffic ,The following bar plots helps to understand how the recipes are distributed and gives an idea of how balanced the dataset is.


checking of data imbalance and outliers but the data is balanced and there is no outliers in the categorical features

![image](image.png) <br>
![image-2](image-2.png)
![image-3](image-3.png)


## Numerical Data are left-skewed
- In order to understand the distribution of the numerical features in our dataset, I used distplot function from seaborn library. This function allows us to visualize the distribution of a given numerical feature by plotting a histogram and fitting a kernel density estimate (KDE) on top of it.

- By using distplot to visualize the distribution of calories, carbohydrate, sugar and protein, I found that all of them are left skewed. This means that the majority of the values are concentrated on the left side of the distribution and there are fewer values on the right side.

- This information is important because it can affect the performance of our models. Left skewness can cause issues with certain statistical tests and models, such as logistic regression, that assume a normal distribution of data. It also indicates that there might be outliers in the data that could have a significant impact on the results of our analysis. Therefore, it is important to keep this skewness in mind when building our models and to consider appropriate preprocessing techniques to address it.

![image-4](image-4.png)
![image-5](image-5.png)


## Examining correlation between features

In order to analyze the correlation between the numerical features of the data, I used the `corr()` method provided by the pandas library. This method calculates the correlation between all pairs of numerical columns in the dataset and returns a correlation matrix. To visualize this matrix, I used the `heatmap()` function from the seaborn library, which creates a heatmap of the correlation values. and `pairplot()` function that creates scatter plots of all pairs of numerical variables in the dataset.

In [14]:
df[['calories', 'carbohydrate', 'sugar', 'protein']].corr()

,calories,carbohydrate,sugar,protein
calories,1.000000,-0.030032,-0.098914,0.182008
carbohydrate,-0.030032,1.000000,0.076842,-0.049172
sugar,-0.098914,0.076842,1.000000,-0.104324
protein,0.182008,-0.049172,-0.104324,1.000000


![image-6](image-6.png)
![image-7](image-7.png)


## No correlation
We can see that there is a relatively weak correlation between the features, with the highest correlation coefficient being 0.18 for the relationship between calories and protein. This suggests that there is not a strong linear relationship between any of the features.

## Measure association with the high traffic

### 1- Nutritions association with hight Traffic

I will use statistical test which is t-test to compare the means of the nutritional information between the high and low traffic recipes, and see if there are significant differences. I will create two separate dataframes: one for high traffic recipes and one for low traffic recipes. Then, I will use the scipy library to perform t-test on each of the nutritional information columns (calories, carbohydrate, protein, sugar) between the two groups. 

It's important to note that t-test is a parametric test and it assumes that the data is normally distributed and that the variances are equal, so if this assumptions are not met, it may not be the best test to use. so I will normalize the features in different data frame to do the test



In [15]:
normalized=normalized=df[['calories','carbohydrate','protein','sugar','high_traffic']]
normalized['calories'] = np.log1p(normalized['calories'])
normalized['carbohydrate'] = np.log1p(normalized['carbohydrate'])
normalized['sugar'] = np.log1p(normalized['sugar'])
normalized['protein'] = np.log1p(normalized['protein'])

high_traffic_df = normalized[normalized['high_traffic'] == 'High']
low_traffic_df = normalized[normalized['high_traffic'] == 'Low']

for i in ['calories','carbohydrate','protein','sugar']:
    t, p = ttest_ind(high_traffic_df[i], low_traffic_df[i])
    print(f"t-statistic for {i}: ", t)
    print(f"p-value for {i}: ", p,'\n')

t-statistic for calories:  2.02972917002691
p-value for calories:  0.04266472851724941 

t-statistic for carbohydrate:  2.0405510365723436
p-value for carbohydrate:  0.04157385862710816 

t-statistic for protein:  4.039791747727636
p-value for protein:  5.7855455152006214e-05 

t-statistic for sugar:  -1.9811420534667563
p-value for sugar:  0.047865784929882506 



it appears the p-values for all features are less than 0.05, which is commonly used as a threshold for determining statistical significance. This indicates that there is a statistically significant difference in means of these features between high traffic and low traffic recipes. so nutritions in the recipe may affect the traffic in some way

### 2- Category association with high traffic

I will use chi-squared test  because it is used to determine if there is a significant association between two categorical variables. In this case, the test is being used to determine if there is a significant association between the category of the recipe and the high traffic.

The contingency table shows the number of recipes in each category that fall into the "High" or "Low" traffic categories. The chi2, p, dof and expected values are calculated from this table. The chi2 value is the test statistic, which measures the deviation of the observed values from the expected values. The p-value is the probability of observing a chi-squared test statistic as extreme or more extreme than the one calculated, under the null hypothesis that the variables are independent.

In [16]:
contingency_table = pd.crosstab(df['category'], df['high_traffic'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
chi2, p

(320.1309861002327, 8.557357582851215e-63)

the chi-squared statistic is 320.1309861002327, and the p-value is 8.557357582851215e-63, which is much less than 0.05, which means that we can reject the null hypothesis and conclude that there is a significant association between the category of the recipe and the level of traffic to the website when the recipe was shown.

as you will see in the following plot that shows the number of recipes that belong to each category and whether they have high traffic or not. From this plot, we can conclude that there is a strong association between the category and high_traffic variables.

![image-8](image-8.png)


Doing the same test wit the servings column

In [17]:
contingency_table = pd.crosstab(df['servings'], df['high_traffic'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
chi2, p

(2.5983468689427824, 0.45777943581507274)

From the following plot and the p-value there is not significant association between the servings and the high traffic data

![image-9](image-9.png)


# New features 

Based on the analysis, it has been determined that there is a strong association between nutritional content, category, and the high traffic feature

for this insights I concluded two thing 

1. that healthy food may play a role in driving traffic that's why the nutritions have significant relation.
2. the difficulty of the recipe may also have affect thats why some categories lead to high traffic than other categories.

As a result, two new features will be extracted: one representing if the recipe is healthy or not and the other representing the difficulty level of the recipe. These features will be used to further enhance the model's predictive capabilities.

## Healthy
after searching I found some general guidelines for interpreting the nutritional information of a recipe:

- Calorie content: A healthy recipe generally has a moderate calorie content. a healthy recipe should have around 400 calories or less.

- Protein content: A healthy recipe should have adequate protein to support muscle growth and repair. a healthy recipe should have at least 10 grams of protein.

- Carbohydrates content: A healthy recipe should have a balance of carbohydrates for energy. a healthy recipe should have around 15-80 grams of carbohydrates.

- Sugar content: A healthy recipe should have minimal added sugar. a healthy recipe should have less than 10 grams of sugar.

- if the category is Vegetable, then the healthy feature is set to "Yes" regardless of the other conditions.



In [18]:
df['healthy'] = np.where((df['calories'] <= 400) & (df['carbohydrate'] >= 15) & (df['carbohydrate'] <= 80) & (df['protein'] >= 10)  & (df['sugar'] <= 10), 'Yes', 'No')
df['healthy'] =df.apply(lambda x: "Yes" if x['category']=='Vegetable' else x["healthy"],axis=1)

df[df['healthy']=='Yes'].value_counts(['high_traffic'])

high_traffic
High            143
Low              48
dtype: int64

we have 191 healthy recipe out of 944 recipe which means 20% of the recipes are healthy.

the probability of getting a high traffic recipe knowing it's healthy is 0.75 which is a very high percentage. 3 out of 4 healthy recipes produces high traffic

![image-10](image-10.png)


![image-16](image-16.png)


## difficuilty

To determine the recipe difficulty we need to take both the category and the servings into consideration.

1. I assigned a certain number of points to each category depending on its difficulty in preparation
2. I will add on this points the number of servings because the more of servings the more difficult the recipe is 

In [19]:
category_points = {'Beverages': 1, 'Breakfast': 1, 'Chicken': 3, 'Chicken Breast': 2, 'Dessert': 4,
                   'Lunch/Snacks': 3, 'Meat': 4, 'One Dish Meal': 3, 'Pork': 4, 'Potato': 1, 'Vegetable': 1}

# Create a dictionary to map servings to difficulty points

# Create a new column "difficulty" by mapping the category and servings to their corresponding points
df['difficulty'] = df['category'].map(category_points) + df['servings']
df[['difficulty','high_traffic']].groupby('difficulty').value_counts()

difficulty  high_traffic
2           Low              39
            High             29
3           High             50
            Low              39
4           High             40
            Low              22
5           High            145
            Low             104
6           High             48
            Low              39
7           High             95
            Low              70
8           High             95
            Low              29
9           High             28
            Low              16
10          High             42
            Low              14
dtype: int64

from these numbers we can't detremine if the difficulty has a significant impact on the high traffic or not so I will do two things.

1. I will use ttest again to measure the association between difficulty and traffic
2. I will divide difficulty into 3 groups to be able to deal with them visually

In [20]:
high_traffic_df = df[df['high_traffic'] == 'High']
low_traffic_df = df[df['high_traffic'] == 'Low']


t, p = ttest_ind(high_traffic_df['difficulty'], low_traffic_df['difficulty'])
print("t-statistic for difficulty: ", t)
print("p-value for difficulty: ", p,'\n')

t-statistic for difficulty:  4.164834969119201
p-value for difficulty:  3.4018057704135366e-05 



It's very obvious that the p-value is less than 0.05, which is commonly used as a threshold for determining statistical significance. This indicates that there is a statistically significant difference in means of these features between high traffic and low traffic recipes. so difficulty of the recipe may affect the traffic in some way

Now I will divide the difficulty into 3 groups: 
- from 2 to 4 is easy
- from 4 to 7 is medium 
- from 7 to 10 is hard

![image-11](image-11.png)


difficult recipes lead to more than 70% chance of getting high traffic

# Model Development 

With a thorough understanding of the data and clear insight into the importance of certain features, the next step is to develop models that can effectively predict high traffic recipes.It is a classification problem so I will try Three different models will be implemented and evaluated, including: 

1. Random Forest
2. Neural Networks
3. Logistic Regression

These models will be trained and tested on the preprocessed and transformed data, in order to determine which one performs the best in terms of accuracy and overall performance.

# Model Evaluation

- To evaluate the performance of the models, I will use the confusion matrix, which is a table that is used to define the performance of a classification algorithm
-  I will use confusion matrix to calculate the accuracy, precision, recall, and F1-score of the models. 
-   I will also use the Receiver Operating Characteristic (ROC) curve, which is a graphical representation of the performance of a classification algorithm, to compare the models

## Prepare the training data 

1. first drops the 'high_traffic', 'recipe', 'difficulty_group', and 'servings' columns
2. then I will convert categorical variables into numerical variables using pandas dummies
3. also I will use the LabelEncoder class to convert the 'high_traffic' column into numerical values. to be a binary problem
4. assign the high_traffic data into target variable and the other data into features variable
5. I will use train_test_split method to split data into training and test data

In case of logistic regression I will need to normalize the numerical variables as Logistic regression is sensitive to scale because it uses the input features to calculate the probability of a certain outcome. If the scale of one feature is much larger than the scale of another feature, it can dominate the other feature and skew the results. Additionally, logistic regression uses the logarithm of the odds ratio, which can also be affected by the scale of the features.


In [21]:
le=LabelEncoder()
features=pd.get_dummies(df.drop(columns=['high_traffic','recipe','servings'],axis=1))
target=le.fit_transform(df['high_traffic'])
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=9)
x_train.shape,x_test.shape

((755, 18), (189, 18))

# Random Forest Classifier

I will use grid search to optimize the model 


In [31]:
rf=RandomForestClassifier()
param_grid = {'n_estimators': [10, 50, 100,150,200],
              'max_depth': [None, 5, 10],
              'random_state': [10,15,20,25]
              }
gs=GridSearchCV(rf,param_grid)
gs.fit(x_train,y_train)


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'n_estimators': [10, 50, 100, 150, 200],
                         'random_state': [10, 15, 20, 25]})

In [32]:
y_pred = gs.predict(x_test)
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("confussion matrix: ",conf_matrix)
print("accuarcy: ",acc)
print("precision: ", precision)
print("recall: ",recall)
print("f1: ",f1)

confussion matrix:  [[100  15]
 [ 22  52]]
accuarcy:  0.8042328042328042
precision:  0.7761194029850746
recall:  0.7027027027027027
f1:  0.7375886524822697


![image-12](image-12.png)


# Random Forest Results

1. accuracy: 80% percent accuracy on test data 
2. True positives : 100  (recipes that were predicted to have high traffic and actually had high traffic)
3. false positives : 15 (recipes that were predicted to have high traffic but actually had low traffic).
4. false negatives : 22 (recipes that were predicted to have low traffic but actually had high traffic)
5. true negative : 52 (recipes that were predicted to have low traffic and actually had low traffic).
6. precision : 0.78 the proportion of correctly predicted positive observations out of all positive predictions
7. recall : 0.70 the proportion of correctly predicted positive observations out of all actual positive observations.
8. f1 : 0.74 It is a measure of the trade-off between precision and recall
9. AUC : 0.85 it is the area under the ROC curve, where the ROC curve is a plot of the true positive rate against the false positive rate

the precision, recall and f1 of the random forest model are relatively high, which indicates that the model is able to accurately identify positive observations and find most of the actual positive observations. Also the AUC value is relatively high which indicate that the model is able to distinguish between positive and negative observations well. we also reached 80% accuarcy 


# Neural Network

In [41]:
model = Sequential()

model.add(Dense(32,input_shape=(18,),activation="relu"))
model.add(Dense(24,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(8,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer='adam',metrics='accuracy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 32)                608       
                                                                 
 dense_11 (Dense)            (None, 24)                792       
                                                                 
 dense_12 (Dense)            (None, 16)                400       
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,945
Trainable params: 1,945
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(x_train,y_train,epochs=300,verbose=1,batch_size=32)

Epoch 1/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6805 - accuracy: 0.5788
Epoch 2/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6621 - accuracy: 0.5987
Epoch 3/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6502 - accuracy: 0.6291
Epoch 4/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6393 - accuracy: 0.6556
Epoch 5/300
24/24 [==============================] - 0s 2ms/step - loss: 0.6285 - accuracy: 0.6649
Epoch 6/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6160 - accuracy: 0.6702
Epoch 7/300
24/24 [==============================] - 0s 1ms/step - loss: 0.6014 - accuracy: 0.6821
Epoch 8/300
24/24 [==============================] - 0s 1ms/step - loss: 0.5828 - accuracy: 0.7099
Epoch 9/300
24/24 [==============================] - 0s 1ms/step - loss: 0.5635 - accuracy: 0.7219
Epoch 10/300
24/24 [==============================] - 0s 2ms/step - loss: 0.5449 - accuracy: 0.7325
Epoch 11/

In [43]:
y_pred =(model.predict(x_test) > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("confussion matrix: ",conf_matrix)
print("accuarcy: ",acc)
print("precision: ", precision)
print("recall: ",recall)
print("f1: ",f1)

6/6 [==============================] - 0s 1ms/step
confussion matrix:  [[94 21]
 [29 45]]
accuarcy:  0.7354497354497355
precision:  0.6818181818181818
recall:  0.6081081081081081
f1:  0.6428571428571428


![download](download.png)


# Neural Results

1. accuracy: 73% percent accuracy on test data 
2. True positives : 94  (recipes that were predicted to have high traffic and actually had high traffic)
3. false positives : 21 (recipes that were predicted to have high traffic but actually had low traffic).
4. false negatives : 29 (recipes that were predicted to have low traffic but actually had high traffic)
5. true negative : 45 (recipes that were predicted to have low traffic and actually had low traffic).
6. precision : 0.68 the proportion of correctly predicted positive observations out of all positive predictions
7. recall : 0.6 the proportion of correctly predicted positive observations out of all actual positive observations.
8. f1 : 0.64 It is a measure of the trade-off between precision and recall
9. AUC : 0.84 it is the area under the ROC curve, where the ROC curve is a plot of the true positive rate against the false positive rate

low precision which indicates that the model is not able to accurately identify positive observations  Also the AUC value is relatively high which indicate that the model is able to distinguish between positive and negative observations well. but we got low accuarcy.

we focus more on the precision in this problem so neural network failed to get us high accuarcy


# Logistic Regression
I need to normalize the numerical data because they are left skewed I will use log transform

In [44]:
le=LabelEncoder()
features=pd.get_dummies(df.drop(columns=['high_traffic','recipe','servings'],axis=1))
target=le.fit_transform(df['high_traffic'])
features['calories'] = np.log1p(features['calories'])
features['carbohydrate'] = np.log1p(features['carbohydrate'])
features['sugar'] = np.log1p(features['sugar'])
features['protein'] = np.log1p(features['protein'])

features

,calories,carbohydrate,sugar,protein,difficulty,category_Beverages,category_Breakfast,category_Chicken,category_Chicken Breast,category_Dessert,category_Lunch/Snacks,category_Meat,category_One Dish Meal,category_Pork,category_Potato,category_Vegetable,healthy_No,healthy_Yes
0,6.640714,3.047945,2.081107,4.212542,10,0,0,0,0,0,0,0,0,1,0,0,1,0
1,3.596764,3.677819,0.506818,0.652325,5,0,0,0,0,0,0,0,0,0,1,0,1,0
2,6.819230,3.776890,1.408545,1.355835,2,0,1,0,0,0,0,0,0,0,0,0,1,0
3,4.585274,3.451890,3.679586,0.019803,5,1,0,0,0,0,0,0,0,0,0,0,1,0
4,3.333989,1.047319,0.587787,0.425268,5,1,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,7.057898,1.842136,3.152309,3.811539,5,0,0,0,0,0,1,0,0,0,0,0,1,0
943,5.746852,4.256010,1.378766,1.925270,3,0,0,0,0,0,0,0,0,0,1,0,1,0
944,6.859342,3.415100,1.519513,2.699346,6,0,0,0,0,0,0,0,0,1,0,0,1,0
945,5.589531,3.604682,0.678034,2.204972,7,0,0,0,0,0,0,0,0,0,1,0,1,0


![image-14](image-14.png)


In [45]:
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=8)
logreg=LogisticRegression()
logreg.fit(x_train,y_train)

LogisticRegression()

In [46]:
y_pred = logreg.predict(x_test)
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("confussion matrix: ",conf_matrix)
print("accuarcy: ",acc)
print("precision: ", precision)
print("recall: ",recall)
print("f1: ",f1)

confussion matrix:  [[94 21]
 [16 58]]
accuarcy:  0.8042328042328042
precision:  0.7341772151898734
recall:  0.7837837837837838
f1:  0.7581699346405228


![image-15](image-15.png)


# Logistic Regression Results

1. accuracy: 80% percent accuracy on test data 
2. True positives : 94  (recipes that were predicted to have high traffic and actually had high traffic)
3. false positives : 21 (recipes that were predicted to have high traffic but actually had low traffic).
4. false negatives : 16 (recipes that were predicted to have low traffic but actually had high traffic)
5. true negative : 58 (recipes that were predicted to have low traffic and actually had low traffic).
6. precision : 0.73 the proportion of correctly predicted positive observations out of all positive predictions
7. recall : 0.78 the proportion of correctly predicted positive observations out of all actual positive observations.
8. f1 : 0.75 It is a measure of the trade-off between precision and recall
9. AUC : 0.86 it is the area under the ROC curve, where the ROC curve is a plot of the true positive rate against the false positive rate

recall and f1 of the logistic regression model are relatively high, but with low precision which indicates that the model is not able to accurately identify positive observations  Also the AUC value is relatively high which indicate that the model is able to distinguish between positive and negative observations well. 

we focus more on the precision in this problem so logistic Regression to get us high accuarcy


# Conclusion

1. Random Forest is the suitable model for us it has accuarcy 80% and can predict the high traffic recipes with precision 78%
2. Healthy Recipes generates More traffic on the site People tends to search for healthy recipes
3. some categories producces high traffic like Vegetable,Pork,Meat and potato
4. people don't care to much about Beverages and breakfast the care more about delicious meals 
5. difficulty of the recipe affect the traffic so the more difficult the recipe is with more servings the more the traffic is.